<a href="https://colab.research.google.com/github/upashanadutta23/DLPROJECT/blob/main/DL_CV_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install hugging face datasets
!pip install datasets
!pip install nibabel
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [ ]:
#Importing necessary Libraries
import nibabel as nib
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from datasets import load_dataset,load_from_disk


In [ ]:
#Processing the scanned Brain Data
def preprocess_nifti(example):
  nii_path = example("nii_filepath")
  #Load the volume data
  vol = nib.load(nii_path).get_fdata()
  #cropping the sub volume
  vol = vol[7:105, 8:132, :108] #(98,124,108)
  #shifting intensities so that it is not negative
  vol = vol + abs(vol.min())
  #normalising it to [0,1]
  vol = vol / vol.max()
  #converting it to torch tensor(1,1,D,H,W)
  t.tensor = torch.from_numpy(vol).unsqueeze(0).unsqueeze(0)
  #scale factor based on y dimension(124 -> 96)
  downsampled = F.interpolate(t_tensor,scale_factor = (scale_factor,scale_factor,scale_factor),mode = "trilinear",align_corners = False)



In [ ]:
#symmetric padding(dim = 96)
_, _, d,h, w = downsampled.shape
pad_d = (96 - d) // 2
pad_h = (96 - h) // 2
pad_w = (96 - w) // 2
padding = (pad_w//2 , pad_w - pad_w//2, pad_h//2, pad_h - pad_h//2, pad_d//2, pad_d - pad_d//2)
final_image = F.pad(downsampled,padding) ##current shape = (1,1,96,96,96)
final_image = final_image.squeeze(0)



NameError: name 'downsampled' is not defined